In [3]:
import ibis
import pandas as pd

## Default Row Limits
It is possible to turn on interactive mode, which automatically executs ibis expressions. By default, ibis limits result sets returned to the local process to 10,000 rows. If you know you require >10000 rows returned, be careful to change the default limit.

## Interactive Mode
Ibis also allows and interactive mode that automatically executes all expressions. This can be useful in a notebook or repl. I personally prefer to epxlicitly execute expresssions, but this is a personal preference.  If you use the interactive mode, I recommnd setting the defaultlimit low to prevent accidentally trying to return an unreasonable number of rows to your local process.

In [4]:
ibis.options.sql.default_limit = None

hdfs_conn = ibis.hdfs_connect(host='bottou03.sjc.cloudera.com')

ibis_conn = ibis.impala.connect(host='bottou01.sjc.cloudera.com',
                                    port=21050,
                                    hdfs_client=hdfs_conn)

In [5]:
pageviews_tbl = ibis_conn.table('wiki_pageviews', database='u_juliet')

What is in a project name? What does this data look like?

In [6]:
project_names_expr = pageviews_tbl.project_name.distinct()
project_names = ibis_conn.execute(project_names_expr)
project_names

0          iu.m.d
1       he.zero.q
2          br.m.q
3          co.m.d
4          zero.f
5          ru.m.b
6             ltg
7       bh.zero.d
8          lv.m.b
9              am
10         vi.m.d
11           ay.d
12           tk.d
13           cr.q
14           is.d
15      iu.zero.d
16           uk.s
17        dv.zero
18        en.zero
19      tl.zero.d
20           kl.d
21           ln.b
22           wa.b
23           da.q
24         sv.m.n
25           la.s
26           sk.s
27             tt
28           ko.q
29             es
          ...    
2367           an
2368           pi
2369       fo.m.s
2370      ha.zero
2371       uz.m.b
2372    kn.zero.s
2373    es.zero.v
2374        sco.m
2375         ks.m
2376       pt.m.q
2377         cy.m
2378       az.m.b
2379       Amanhã
2380          21º
2381          pnb
2382        wg-en
2383         pt.b
2384           be
2385    vi.zero.q
2386       qu.m.q
2387         wo.q
2388         ar.d
2389       uk.m.b
2390    wa.zero.d
2391      

Maybe we can understand this by finding the projects with the most pages. Let's group by porject name and then count the size of the groups.

In [7]:
project_page_counts = pageviews_tbl.group_by(pageviews_tbl.project_name)\
                                   .size()\
                                   .sort_by(('count', False))
project_names = ibis_conn.execute(project_page_counts)

To find something interesting, it'll help to understand the language. 

In [8]:
[name for name in project_names if 'en' in name]

[]

The part of the project name after the '.' specifies a special type of wiki. Let's just look at the standard wiki pages (ie, not media-wiki) that are also written in English.

In [9]:
ibis_conn.execute(pageviews_tbl[pageviews_tbl.project_name == 'en'].limit(10))

,project_name,page_name,n_views,n_bytes,day,hour,month,year
0,en,!!!,3,0,01,07,01,2016
1,en,!Bastardos_en_Vivo!,1,0,01,07,01,2016
2,en,!Kung_language,1,0,01,07,01,2016
3,en,!Oka_Tokat,1,0,01,07,01,2016
4,en,!T.O.O.H.!,1,0,01,07,01,2016
5,en,"""@""_(album)",1,0,01,07,01,2016
6,en,"""C""_Is_for_(Please_Insert_Sophomoric_Genitalia...",1,0,01,07,01,2016
7,en,"""Crocodile""_Dundee",6,0,01,07,01,2016
8,en,"""Crocodile""_Dundee_II",1,0,01,07,01,2016
9,en,"""Dr._Death""_Steve_Williams",6,0,01,07,01,2016


Project_name is homogenous in this dataset, so lets just take the projection of all other columns.

In [10]:
en_pageviews = pageviews_tbl[pageviews_tbl.project_name == 'en'].projection(['page_name',
                                                                              'n_views',
                                                                             'n_bytes',
                                                                             'day',
                                                                             'hour',
                                                                             'month',
                                                                             'year'])

In [11]:
ibis_conn.execute(en_pageviews.limit(10))

,page_name,n_views,n_bytes,day,hour,month,year
0,!,1,0,01,05,01,2016
1,!!!,2,0,01,05,01,2016
2,!!!Fuck_You!!!,1,0,01,05,01,2016
3,!!!Fuck_You!!!_And_Then_Some,1,0,01,05,01,2016
4,!Action_Pact!,1,0,01,05,01,2016
5,!Kung_San_people,1,0,01,05,01,2016
6,!Kung_people,2,0,01,05,01,2016
7,!T.O.O.H.!,1,0,01,05,01,2016
8,!WOWOW!,1,0,01,05,01,2016
9,"""...And_Ladies_of_the_Club""",1,0,01,05,01,2016


It seems that we should exclude these pages with no names, and NaN counts. (With big data sets, you will find all
types of messed up data.)

In [12]:
top_10_pg_views_hourly = en_pageviews.sort_by((en_pageviews.n_views, False)).limit(10)
ibis_conn.execute(top_10_pg_views_hourly)

,page_name,n_views,n_bytes,day,hour,month,year
0,,NaN,NaN,31,01,12,2015
1,http://example.com/?,NaN,NaN,31,01,12,2015
2,,NaN,NaN,31,01,12,2015
3,,NaN,NaN,31,01,12,2015
4,,NaN,NaN,31,01,12,2015
5,Special:Export/,1936066.0,0.0,31,08,12,2015
6,Special:Export/,1121921.0,0.0,31,09,12,2015
7,Main_Page,724796.0,0.0,02,09,01,2016
8,Main_Page,707561.0,0.0,02,07,01,2016
9,Main_Page,655542.0,0.0,02,08,01,2016


In [13]:
null_pg_views = en_pageviews[en_pageviews.n_views.isnull()]

In [14]:
ibis_conn.execute(null_pg_views)

,page_name,n_views,n_bytes,day,hour,month,year
0,,NaN,NaN,31,01,12,2015
1,,NaN,NaN,31,01,12,2015
2,,NaN,NaN,31,01,12,2015
3,,NaN,NaN,31,01,12,2015
4,http://example.com/?,NaN,NaN,31,01,12,2015


In [15]:
nn_pg_views = en_pageviews[en_pageviews.n_views.notnull()]

What are the top ten page in this series that 

In [16]:
ibis_conn.execute(nn_pg_views.sort_by((nn_pg_views.n_views, False)).limit(10))

,page_name,n_views,n_bytes,day,hour,month,year
0,Special:Export/,1936066,0,31,08,12,2015
1,Special:Export/,1121921,0,31,09,12,2015
2,Main_Page,724796,0,02,09,01,2016
3,Main_Page,707561,0,02,07,01,2016
4,Main_Page,655542,0,02,08,01,2016
5,Main_Page,650881,0,03,07,01,2016
6,Main_Page,649290,0,03,08,01,2016
7,Main_Page,647347,0,02,06,01,2016
8,Main_Page,645168,0,30,09,12,2015
9,Main_Page,641259,0,30,08,12,2015


hangover, brands of champagne, mew years traditions, time differences, international datetime,

In [17]:
champagne_df = ibis_conn.execute(nn_pg_views[nn_pg_views.page_name.lower() == 'champagne'])

In [18]:
champagne_df.sort(['day', 'hour'])

/Users/juliet/.virtualenvs/pyhadoop/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,page_name,n_views,n_bytes,day,hour,month,year
18,Champagne,223,0,01,00,01,2016
25,Champagne,176,0,01,01,01,2016
41,Champagne,186,0,01,02,01,2016
17,Champagne,162,0,01,03,01,2016
3,Champagne,147,0,01,04,01,2016
24,Champagne,150,0,01,05,01,2016
2,Champagne,226,0,01,06,01,2016
20,Champagne,210,0,01,07,01,2016
6,Champagne,113,0,01,08,01,2016
29,Champagne,117,0,01,09,01,2016


In [30]:
champagne_df['time'] = pd.to_datetime(champagne_df[['year', 'month', 'day', 'hour']])

In [ ]:
champagne_df[['n_views', 'time']].plot()

In [28]:
w_daily_views = nn_pg_views.group_by(['page_name', 'month', 'day']).aggregate(daily_views=nn_pg_views.n_views.sum())

ibis_conn.execute(w_daily_views.sort_by((w_daily_views.daily_views, False)).limit(10))

,page_name,month,day,daily_views
0,Main_Page,01,02,10189772
1,Main_Page,12,30,10113400
2,Main_Page,01,03,10005506
3,Main_Page,12,31,9772903
4,Main_Page,01,01,9011875
5,Special:Export/,12,31,3630690
6,List_of_stock_market_crashes_and_bear_markets,01,01,1881837
7,List_of_stock_market_crashes_and_bear_markets,01,02,1797907
8,List_of_stock_market_crashes_and_bear_markets,01,03,1751699
9,Special:Search,12,30,1390842


In [29]:
tot_view = nn_pg_views.group_by('page_name').aggregate(all_views=nn_pg_views.n_views.sum())
ibis_conn.execute(tot_view.sort_by((tot_view.all_views, False)).limit(30))

,page_name,all_views
0,Main_Page,49093456
1,Special:Search,6747343
2,List_of_stock_market_crashes_and_bear_markets,5529739
3,Special:Export/,3631060
4,Special:Book,1831484
5,Web_scraping,859476
6,-,753519
7,Star_Wars:_The_Force_Awakens,722752
8,Java_(programming_language),591223
9,Pincushion,411923
